In [31]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval
import random

In [6]:
%run MODEL_LIST.ipynb            #Load the models

In [7]:
df = pd.read_csv("../data/ufc-master.csv")

In [8]:
df.head()

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1
0,Alistair Overeem,Walt Harris,135.0,-155.0,135.000000,64.516129,5/16/2020,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.0,2.54,-7.62,-3.0,-0.69,-0.7,-1.08,1.0,1.0
1,Claudia Gadelha,Angela Hill,-255.0,215.0,39.215686,215.000000,5/16/2020,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.0,-2.54,2.54,4.0,2.36,-0.5,-2.66,1.0,1.0
2,Dan Ige,Edson Barboza,112.0,-132.0,112.000000,75.757576,5/16/2020,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.0,10.16,10.16,6.0,-0.04,-0.6,-1.69,1.0,1.0
3,Krzysztof Jotko,Eryk Anders,-105.0,-105.0,95.238095,95.238095,5/16/2020,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.0,0.00,-5.08,3.0,0.04,0.0,0.07,1.0,1.0
4,Song Yadong,Marlon Vera,-200.0,170.0,50.000000,170.000000,5/16/2020,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.0,0.00,7.62,5.0,-0.62,0.8,0.15,1.0,1.0


In [9]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [10]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [11]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [12]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[:-200]
odds_train = odds_df[:-200]
label_train = label_df[:-200]

df_test = df[-200:]
odds_test = odds_df[-200:]
label_test = label_df[-200:]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4000
4000
4000


In [13]:
def get_ev(input_df, input_model, input_features, input_labels, odds_input, min_ev = 0, verbose=False, get_total=False):
    df_sel = input_df[input_features]
    df_sel = df_sel.dropna()
    df_sel = pd.get_dummies(df_sel)
    labels_sel = input_labels[input_labels.index.isin(df_sel.index)]
    odds_sel = odds_input[odds_input.index.isin(df_sel.index)] 
    best_score = custom_cv_eval(df_sel, input_model, labels_sel, odds_sel, min_ev = min_ev, verbose=verbose, 
                                get_total=get_total)
    return best_score

In [45]:
def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
                  test_odds, verbose=True):
    model_score = 0
    
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    
    
    
    display(df_train)
    #display(df_test)
    display(df_test)
    
    
    return(model_score)

In [46]:
m1_score = evaluate_model(model_2, features_2, min_ev_2, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

print(m1_score)

,R_Stance,B_fighter,B_avg_TD_pct
4072,Southpaw,Michael Johnson,NaN
4073,Orthodox,Igor Pokrajac,0.553333
4074,Orthodox,Demian Maia,0.405556
4075,Southpaw,Rick Story,0.742000
4076,Orthodox,Leonard Garcia,0.000000
...,...,...,...
4268,Orthodox,Darren Elkins,NaN
4269,Orthodox,Daniel Roberts,NaN
4270,Orthodox,Chase Gormley,1.000000
4271,Orthodox,Julio Paulino,NaN


,B_avg_TD_pct,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch,B_fighter_Aaron Riley,B_fighter_Aaron Rosa,B_fighter_Aaron Simpson,B_fighter_Abdul Razak Alhassan,B_fighter_Abel Trujillo,...,B_fighter_Yushin Okami,B_fighter_Yves Edwards,B_fighter_Yves Jabouin,B_fighter_Zabit Magomedsharipov,B_fighter_Zach Makovsky,B_fighter_Zak Cummings,B_fighter_Zak Ottow,B_fighter_Zarah Fairn,B_fighter_Zhang Weili,B_fighter_Zubaira Tukhugov
0,0.330000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.370000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.440000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.340000,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.420000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4064,0.475714,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4065,0.674000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4066,0.157143,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4067,0.222222,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,B_avg_TD_pct,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch,B_fighter_Aaron Riley,B_fighter_Aaron Rosa,B_fighter_Aaron Simpson,B_fighter_Abdul Razak Alhassan,B_fighter_Abel Trujillo,...,B_fighter_Yushin Okami,B_fighter_Yves Edwards,B_fighter_Yves Jabouin,B_fighter_Zabit Magomedsharipov,B_fighter_Zach Makovsky,B_fighter_Zak Cummings,B_fighter_Zak Ottow,B_fighter_Zarah Fairn,B_fighter_Zhang Weili,B_fighter_Zubaira Tukhugov
4073,0.553333,0,1,0,0,0.0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4074,0.405556,0,1,0,0,0.0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4075,0.742000,0,0,1,0,0.0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4076,0.000000,0,1,0,0,0.0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4082,0.150000,0,1,0,0,0.0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4265,0.000000,0,1,0,0,0.0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4266,0.662500,0,1,0,0,0.0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4267,0.480000,0,1,0,0,0.0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4270,1.000000,0,1,0,0,0.0,0.0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


0
